In [1]:
# !unzip /home/s2120006/TFS/vision/dataset/OneHand_bef_stack.zip
print('done')

done


In [1]:
import numpy as np
from PIL import Image
import os
from tensorflow.keras.utils import load_img
#from keras.preprocessing.image import load_img
from tensorflow.keras.utils import img_to_array
# from keras.preprocessing.image import img_to_array
import shutil
#from keras.preprocessing.sequence import pad_sequences
import pickle
#from random import randint
import random
import shutil
#import extcolors
#from colormap import rgb2hex
import h5py
import pandas as pd

2022-12-13 20:40:16.915309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 20:40:19.425013: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/s2120006/miniconda3/lib
2022-12-13 20:40:19.425562: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/s2120006/miniconda3/lib
2022-12-13 20:40:19.425570: W tensorflow/compiler/tf2tensorrt/utils/py

In [2]:
base_path = '/home/s2120006/TFS/vision/dataset/OneHand/'

In [4]:
#### stack photo and label for LSTM (5 inputs)

# num_sub, num_mainfile, subDir, num_files, allfiles, crp_count, crop_all_folder, padding_file, sub_crop_count  = 0,0,0,0,0, 0,0,0,0
from numpy import asarray
crp_count=0
photos = []
# seq_name = list()
labels = []
count, subDir, num_mainfile = 0, 0, 0
for subtype_folder in os.listdir(base_path):
            # num_sub += 1
            subtype_path = base_path + subtype_folder + '/'


            for mainfile_folder in os.listdir(subtype_path):
                num_mainfile += 1
                mainfile_path = subtype_path + mainfile_folder + '/'


                # print(mainfile_folder)
                for filename in os.listdir(mainfile_path):
                    if (filename != '.DS_Store') and (filename != 'desktop.ini'):
                        groupFolder = mainfile_path + filename + '/Humand hand/'
                        subDir += 1
                        count += 1
                        # print(filename)
                        # print(groupFolder)
                        for crp_folder in os.listdir(groupFolder):
                            crp_path = groupFolder + crp_folder + '/'
                            crp_count += 1
                            for padding_folder in os.listdir(crp_path):
                                if padding_folder[:3] == 'pad' or padding_folder[:3] == 'dum':
                                    padding_path = crp_path + padding_folder + '/'
                                    for img_files in os.listdir(padding_path):
                                        # img_path = padding_path + img_files
                                        # print(img_path)

                                        try:
                                            img_path = padding_path + img_files
                                            photo = load_img(img_path,target_size=(224,224))
                                            photo = img_to_array(photo)
                                            photos.append(photo)

                                        except FileNotFoundError:
                                            print('File Error:=>  ', img_path)

                        labels.append(mainfile_folder)
                        # print(count)

            print(num_mainfile)
                # labels.append(mainfile_folder)
# data = asarray(photos)
# labels = asarray(labels)
# print(data.shape, labels.shape)
# print(labels)

photos = np.stack(photos)
labels = np.stack(labels)
datas = photos.reshape(count,45,224,224,3)
print(datas.shape, labels.shape)
print(set(labels))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
(3398, 45, 224, 224, 3) (3398,)
{'6', 'ป', 'อิ', '3', 'ง', '1', 'ม', 'ฏ', '5', 'ต', '2', '4', 'ค', '7', 'บ', 'ณ', 'ถ', 'พ', 'ก', '8', 'ฝ', 'ท', 'ฐ', '40', '80', 'ฆ', 'ร', 'จ', 'อื(คือ)', 'ฬ', 'ฉ', '90', '20', '100', 'ฒ', '100,000', 'อี', 'ด', 'อ', 'ผ', 'ฑ', 'ซ', '0', 'ฌ', 'ใอ', 'น', 'ธ', 'ษ', 'ฮ', '1,000', 'ไอ', 'อึ(นึก)', 'ภ', 'ล', 'ย', '70', 'ข', '50', '10,000', 'ช', 'ศ', 'ส', 'โอ', 'ญ', 'ฟ', '9', 'ว', 'ห', '1,000,000', '30', '10', '60', 'ฎ'}


In [5]:
#### stack photo and label for general models (4 inputs)


# num_sub, num_mainfile, subDir, num_files, allfiles, crp_count, crop_all_folder, padding_file, sub_crop_count  = 0,0,0,0,0, 0,0,0,0
from numpy import asarray
crp_count=0
photos = []
# seq_name = list()
labels = []
count, subDir, num_mainfile = 0, 0, 0
for subtype_folder in os.listdir(base_path):
            # num_sub += 1
            subtype_path = base_path + subtype_folder + '/'


            for mainfile_folder in os.listdir(subtype_path):
                num_mainfile += 1
                mainfile_path = subtype_path + mainfile_folder + '/'


                # print(mainfile_folder)
                for filename in os.listdir(mainfile_path):
                    if (filename != '.DS_Store') and (filename != 'desktop.ini'):
                        groupFolder = mainfile_path + filename + '/Humand hand/'
                        subDir += 1
                        count += 1
                        # print(filename)
                        # print(groupFolder)
                        for crp_folder in os.listdir(groupFolder):
                            crp_path = groupFolder + crp_folder + '/'
                            crp_count += 1
                            for padding_folder in os.listdir(crp_path):
                                if padding_folder[:3] == 'pad' or padding_folder[:3] == 'dum':
                                    padding_path = crp_path + padding_folder + '/'
                                    for img_files in os.listdir(padding_path):
                                        # img_path = padding_path + img_files
                                        # print(img_path)
                                        count += 1
                                        try:
                                            img_path = padding_path + img_files
                                            photo = load_img(img_path,target_size=(224,224))
                                            photo = img_to_array(photo)
                                            photos.append(photo)
                                            labels.append(mainfile_folder)
                                            
                                        except FileNotFoundError:
                                            print('File Error:=>  ', img_path)

                        
                            # print(count)

                # print(num_mainfile)
                # labels.append(mainfile_folder)
                
photos = asarray(photos)
labels = asarray(labels)
datas = photos.reshape(count, 224, 224, 3)
print(data.shape, labels.shape)
print(labels)

# photos = np.stack(photos)
# labels = np.stack(labels)
# datas = photos.reshape(count,45,224,224,3)
# print(datas.shape, labels.shape)
print(set(labels))

np.savez_compressed('/home/s2120006/TFS/vision/dataset/Onehand_Stack_4inputs',data = datas, label = labels)
print('******** Done *********')

ValueError: cannot reshape array of size 23017236480 into shape (156308,224,224,3)

In [5]:
# from datetime import datetime
# current_dateTime = datetime.now()
# current_dateTime = current_dateTime.strftime("%Y-%m-%d_%H-%M-%S")
# print(current_dateTime)

In [6]:
np.savez_compressed('/home/s2120006/TFS/vision/dataset/Onehand_Stack',data = datas, label = labels)
print('******** Done *********')

******** Done *********


In [7]:
# np.save('/home/s2120006/TFS/vision/label',labels)

In [8]:
# pre_data = datas
# f = open(f'Data_LSTM_OneHand.pkl','wb')
# pickle.dump(pre_data,f)
# f.close()

# print('*******************Done***************')

In [9]:
# pre_labels = labels
# g = open(f'labels_LSTM_OneHand.pkl','wb')
# pickle.dump(pre_labels, g)
# g.close()